# load data

In [1]:
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import os.path as osp
from tqdm import tqdm

In [105]:
N = 6111
# N = 10
dir_name = './data/'
raw_datasets = []
for i in tqdm(range(N)):
	raw_data = torch.load(osp.join(dir_name,f'data_{i}.pt'))
	raw_datasets.append(raw_data)

100%|██████████| 6111/6111 [01:33<00:00, 65.09it/s] 


In [106]:
def restrict_to_one_feature_col(raw_dataset, feature_col=1):
	dataset = []
	for raw_data in tqdm(raw_dataset):
		data = raw_data.clone()
		edge_mask = raw_data.edge_attr[:,feature_col].nonzero()[0]
		data.edge_index = raw_data.edge_index[:,edge_mask]
		del(data.edge_attr)
		dataset.append(data)
	return dataset

In [107]:
datasets = restrict_to_one_feature_col(raw_datasets, feature_col=2)

100%|██████████| 6111/6111 [00:07<00:00, 813.38it/s] 


## train-test split

In [110]:
train_idx = list()
test_name = list()
test_idx = list()
y_train = list()
with open('./data/raw/graph_labels.txt', 'r') as f:
    for i,line in enumerate(f):
        t = line.split(',')
        if len(t[1][:-1]) == 0:
            test_name.append(t[0])
            test_idx.append(i)
        else:
            train_idx.append(i)
            y_train.append(int(t[1][:-1]))

In [111]:
len(train_idx),len(test_idx)

(4888, 1223)

In [112]:
train_val_set = [datasets[i] for i in train_idx]
test_set = [datasets[i] for i in test_idx]

sequential embedding contains a beginning and an ending

In [113]:
train_val_set[0].x.shape

torch.Size([185, 86])

## add label

In [114]:
for data, y in zip(train_val_set, y_train):
	data.y = y

## train-valid split

In [115]:
train_set, val_set = torch.utils.data.random_split(train_val_set, [4400,488], generator=torch.Generator().manual_seed(42))

In [116]:
len(train_set), len(val_set)

(4400, 488)

## Hyperparameters

In [141]:
import numpy as np
num_node_features = train_set[0].num_node_features # 86
print(num_node_features)
num_classes = len(np.unique(y_train)) # 18
print(num_classes)
lr = 1e-3
epochs = 100
batch_size = 128
hidden_dim = 64

86
18


## Dataloader

In [142]:
from torch.utils.data.dataloader import default_collate
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
collate_fn = lambda x: tuple(x_.to(device) for x_ in default_collate(x))
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

In [143]:
train_val_loader =DataLoader(train_val_set, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

In [144]:
for step, data in enumerate(train_loader):
	print(f'Step {step + 1}:')
	print('=======')
	print(f'Number of graphs in the current batch: {data.num_graphs}')
	print(data.is_cuda)
	print()
	break

Step 1:
Number of graphs in the current batch: 128
False



In [121]:
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.nn as gnn
from torch_geometric.utils import dropout_adj

In [129]:
def train(model, optimizer, criterion):
	model.train()
	for data in tqdm(train_loader):
		data = data.to(device)
		out,_ = model(data.x, data.edge_index, data.batch)
		loss = criterion(out, data.y)
		loss.backward()
		optimizer.step()
		optimizer.zero_grad()

def eval(model, loader, criterion):
	model.eval()
	correct = 0
	for data in loader:
		data = data.to(device)
		out,_ = model(data.x, data.edge_index, data.batch)
		loss = criterion(out, data.y)
		pred = out.argmax(dim=1)
		correct += int((pred == data.y).sum())
		acc = correct / len(loader.dataset)
	return acc, loss

def predict(model):
	pred_proba = []
	model.eval()
	for data in test_loader:
		data = data.to(device)
		out,_ = model(data.x, data.edge_index, data.batch)
		pred_proba.append(out.detach().cpu().numpy())
		del out
		torch.cuda.empty_cache()
	pred_proba = np.vstack(pred_proba)
	pred_proba = np.exp(pred_proba)
	return pred_proba

def embed(model, loader):
	embeddings = []
	model.eval()
	for data in loader:
		data = data.to(device)
		_, embedding = model(data.x, data.edge_index, data.batch)
		embeddings.append(embedding.detach().cpu().numpy())
		del embedding
		torch.cuda.empty_cache()
	embeddings = np.vstack(embeddings)
	return embeddings

### load & save checkpoint 

In [18]:
import os.path as osp
def load_checkpoint(model, optimizer, path):
	if osp.exists(path):
		checkpoint = torch.load(path)
		model.load_state_dict(checkpoint['model_state_dict'])
		optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
		print('checkpoint loaded')
		return checkpoint.get('loss')

In [19]:
def save_checkpoint(model, optimizer, loss, path):
	torch.save({
							'epoch': epochs,
							'model_state_dict': model.state_dict(),
							'optimizer_state_dict': optimizer.state_dict(),
							'loss': loss,
							}, path)

### train epochs and save checkpoints

In [123]:
import time 
def train_epochs(model, optimizer, criterion, path, patience=10, from_scratch=True):
	checkpoint_loss = None
	if not from_scratch:
		checkpoint_loss = load_checkpoint(model, optimizer, path=path)
	best_val_loss = checkpoint_loss if checkpoint_loss else 2.8 #2.8 is the loss of equi-ignorant guess
	best_val_acc = 0
	for epoch in range(epochs):
		train(model, optimizer, criterion)
		train_acc, train_loss = eval(model, train_loader, criterion)
		val_acc, val_loss = eval(model, val_loader, criterion)
		print(f'Epoch: {epoch:03d}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')
		if val_loss < best_val_loss:
			save_checkpoint(model, optimizer, loss=val_loss, path=path)
			best_val_loss = val_loss
			print('checkpoint saved')
			patience = patience
		else:
			patience -= 1
			if patience == 0:
				print('early stopping')
				break

In [124]:
def train_epochs_full(model, optimizer, criterion, path, patience=10, from_scratch=True):
	checkpoint_loss = None
	if not from_scratch:
		checkpoint_loss = load_checkpoint(model, optimizer, path=path)
	best_train_loss = checkpoint_loss if checkpoint_loss else 2.8 #2.8 is the loss of equi-ignorant guess
	best_train_acc = 0
	for epoch in range(epochs):
		train(model, optimizer, criterion)
		train_acc, train_loss = eval(model, train_val_loader, criterion)
		time.sleep(0.5)
		print(f'Epoch: {epoch:03d}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
		if train_loss < best_train_loss:
			save_checkpoint(model, optimizer, loss=train_loss, path=path)
			best_train_loss = train_loss
			print('checkpoint saved')
			patience = patience
		else:
			patience -= 1
			if patience == 0:
				print('early stopping')
				break

### write

In [22]:
import csv
def write_csv(fname, pred_proba):
    with open(f'./predictions/{fname}.csv', 'w') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        lst = list()
        for i in range(18):
            lst.append('class'+str(i))
        lst.insert(0, "name")
        writer.writerow(lst)
        for i, protein in enumerate(test_name):
            lst = pred_proba[i,:].tolist()
            lst.insert(0, protein)
            writer.writerow(lst)

In [23]:
import gc
def clean_gpu_memory():
	gc.collect()
	torch.cuda.empty_cache()

# GNN2: GIN

In [132]:
class GIN_2(torch.nn.Module):
    def __init__(self, hidden_channels, edge_feature_col=1):
        super(GIN_2, self).__init__()
        torch.manual_seed(12345)
        self.p = 0.2
        self.feature_col = edge_feature_col
        self.bn = gnn.BatchNorm(hidden_channels)
        self.gin = gnn.GIN(in_channels=num_node_features, 
                            hidden_channels=hidden_channels,
                            num_layers=2,
                            dropout= 0.2,
                            norm=self.bn,
                            )
        self.lin = gnn.MLP(in_channels=hidden_channels, 
                            hidden_channels=hidden_channels,
                            out_channels=num_classes, 
                            num_layers=2, 
                            dropout=0.2)

    def forward(self, x, edge_index, batch):
        x = self.gin(x=x, edge_index=edge_index)
        x = gnn.global_add_pool(x, batch)
        x = self.bn(x)
        x = x.relu()
        emb = x
        x = self.lin(x)
        
        return F.log_softmax(x, dim=1), emb

In [147]:
model_gin = GIN_2(hidden_channels=hidden_dim).to(device)
print(model_gin)
optimizer_1 = torch.optim.Adam(model_gin.parameters(),lr=1e-4,weight_decay=0.01)
criterion = nn.CrossEntropyLoss()

GIN_2(
  (bn): BatchNorm(64)
  (gin): GIN(86, 64, num_layers=2)
  (lin): MLP(64, 64, 18)
)


### Train

In [148]:
train_epochs(model_gin, optimizer_1, criterion, path='model_gin.pt', patience=50, from_scratch=True)

100%|██████████| 35/35 [00:02<00:00, 16.57it/s]


Epoch: 000, Train Loss: 2.8334, Train Acc: 0.1332, Val Loss: 2.8648, Val Acc: 0.1270


100%|██████████| 35/35 [00:01<00:00, 20.73it/s]


Epoch: 001, Train Loss: 2.9140, Train Acc: 0.2139, Val Loss: 2.8042, Val Acc: 0.2090


100%|██████████| 35/35 [00:01<00:00, 20.60it/s]


...

100%|██████████| 35/35 [00:01<00:00, 22.23it/s]


Epoch: 062, Train Loss: 1.8857, Train Acc: 0.3802, Val Loss: 2.1267, Val Acc: 0.3586


100%|██████████| 35/35 [00:01<00:00, 21.92it/s]


Epoch: 063, Train Loss: 1.9554, Train Acc: 0.4000, Val Loss: 2.0670, Val Acc: 0.3545
early stopping


: 

In [24]:
#train_epochs_full(model_gin, optimizer_1, path='model_gin.pt', patience=50, from_scratch=True)

## Load and write results

In [25]:
#load_checkpoint(model_gin, optimizer_1, path='./model_gin.pt')
#write_csv('model_gin',predict(model_gin))

## 

In [26]:
from torch.nn import Linear, Sequential, BatchNorm1d, ReLU, Dropout
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GINConv
from torch_geometric.nn import global_mean_pool, global_add_pool

In [31]:
class GIN(torch.nn.Module):
    """GIN"""
    def __init__(self, dim_h):
        super(GIN, self).__init__()
        self.conv1 = GINConv(
            Sequential(Linear(num_node_features, dim_h),
                       BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()))
        self.conv2 = GINConv(
            Sequential(Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()))
        self.conv3 = GINConv(
            Sequential(Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()))
        self.lin1 = Linear(dim_h*3, dim_h*3)
        self.lin2 = Linear(dim_h*3, num_classes)

    def forward(self, x, edge_index, edge_attr, batch):
        # Node embeddings 
        h1 = self.conv1(x, edge_index)
        h2 = self.conv2(h1, edge_index)
        h3 = self.conv3(h2, edge_index)

        # Graph-level readout
        h1 = global_add_pool(h1, batch)
        h2 = global_add_pool(h2, batch)
        h3 = global_add_pool(h3, batch)

        # Concatenate graph embeddings
        h = torch.cat((h1, h2, h3), dim=1)

        # Classifier
        h = self.lin1(h)
        h = h.relu()
        h = F.dropout(h, p=0.5, training=self.training)
        h = self.lin2(h)
        
        return h, F.log_softmax(h, dim=1)

In [37]:
gin = GIN(dim_h=32).to(device)
optimizer_1 = torch.optim.Adam(gin.parameters(),lr=lr,weight_decay=0.01)
criterion = nn.CrossEntropyLoss()

In [35]:
print(device)

cuda


In [38]:
train_epochs(gin, optimizer_1, path='model_gin2.pt', patience=50, from_scratch=True)

100%|██████████| 138/138 [00:03<00:00, 45.89it/s]


Epoch: 000, Train Loss: 2.4834, Train Acc: 0.1934, Val Loss: 2.6702, Val Acc: 0.1516
checkpoint saved


100%|██████████| 138/138 [00:01<00:00, 82.65it/s]


Epoch: 001, Train Loss: 2.3889, Train Acc: 0.2645, Val Loss: 2.2086, Val Acc: 0.2541
checkpoint saved


100%|██████████| 138/138 [00:01<00:00, 81.75it/s]


...

Epoch: 053, Train Loss: 2.2509, Train Acc: 0.4314, Val Loss: 1.9488, Val Acc: 0.4180


100%|██████████| 138/138 [00:01<00:00, 83.33it/s]


Epoch: 054, Train Loss: 1.7103, Train Acc: 0.4495, Val Loss: 2.1500, Val Acc: 0.4037


100%|██████████| 138/138 [00:01<00:00, 81.35it/s]


Epoch: 055, Train Loss: 2.0469, Train Acc: 0.4786, Val Loss: 2.3809, Val Acc: 0.4857
early stopping


In [ ]:
#train_epochs_full(model_gin, optimizer_1, path='model_gin2.pt', patience=50, from_scratch=True)